In [410]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import plotly.express as px
import warnings


In [411]:
df=pd.read_csv('sales.csv')

In [412]:
df.sample(5)

,Row ID,Order ID,Order Date Fa,Ship Date Fa,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
7313,7314,US-2022-129910,1402/04/29,1402/05/02,Standard Class,ME-17320,Maria Etezadi,Home Office,United States,Columbus,...,31907,South,OFF-ST-10003282,Office Supplies,Storage,"Advantus 10-Drawer Portable Organizer, Chrome Metal Frame, Smoke Drawers",59.760,1,0.0,16.7328
1631,1632,US-2019-159926,1399/12/19,1399/12/23,Standard Class,CS-11950,Carlos Soltero,Consumer,United States,Philadelphia,...,19140,East,TEC-PH-10001128,Technology,Phones,Motorola Droid Maxx,539.964,6,0.4,-107.9928
2877,2878,US-2020-124044,1400/08/21,1400/08/24,Second Class,MS-17830,Melanie Seite,Consumer,United States,Rochester,...,14609,East,OFF-BI-10003460,Office Supplies,Binders,Acco 3-Hole Punch,10.512,3,0.2,3.6792
9407,9408,US-2022-158561,1402/12/12,1402/12/17,Second Class,BB-11545,Brenda Bowman,Corporate,United States,Fort Lauderdale,...,33311,South,OFF-AP-10002651,Office Supplies,Appliances,Hoover Upright Vacuum With Dirt Cup,1158.120,5,0.2,130.2885
5241,5242,US-2021-157763,1401/08/16,1401/08/21,Standard Class,KH-16330,Katharine Harms,Corporate,United States,Bowling Green,...,42104,South,FUR-CH-10000988,Furniture,Chairs,Hon Olson Stacker Stools,140.810,1,0.0,39.4268


In [413]:
df.duplicated().sum()

0

In [414]:
df1 = df.sort_values(by='Order ID', ascending=True)[['Order ID', 'Customer ID', 'Product ID', 'Product Name', 'Category', 'Sub-Category']]
df_order_count = df1['Order ID'].value_counts()  

# Filter Order IDs that appear at least 3 times  
order_ids_to_keep = df_order_count[df_order_count >= 3].index  

# Filter the DataFrame to only include these Order IDs  
filtered_df1 = df1[df1['Order ID'].isin(order_ids_to_keep)].copy()

In [415]:
Products = filtered_df1[['Product ID', 'Product Name', 'Category']].drop_duplicates().set_index('Product ID')
display(Products.head(5))

,Product Name,Category
Product ID,,
FUR-FU-10001025,"Eldon Imàge Series Desk Accessories, Clear",Furniture
FUR-FU-10002445,"DAX Two-Tone Rosewood/Black Document Frame, Desktop, 5 x 7",Furniture
FUR-FU-10002960,"Eldon 200 Class Desk Accessories, Burgundy",Furniture
OFF-PA-10002689,Weyerhaeuser First Choice Laser/Copy Paper (20Lb. and 88 Bright),Office Supplies
TEC-PH-10001300,"iKross Bluetooth Portable Keyboard + Cell Phone Stand Holder + Brush for Apple iPhone 5S 5C 5, 4S 4",Technology


In [416]:
order_producs = pd.crosstab(filtered_df1['Order ID'], filtered_df1['Product ID'])

In [417]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(order_producs)

NearestNeighbors(algorithm='brute', metric='cosine')

In [418]:
def find_categories(input_df, categories_list):
    try:
        for col in input_df.columns:  
            for idx, row in input_df.iterrows():  # Use iterrows to allow row modification  
                if row[col] > 0 and Products.loc[col, 'Category'] in categories_list:  
                    input_df.at[idx, col] += 1  # Update the DataFrame directly  
    except:
        return input_df
    return input_df

def recommend_item(order_id, n_recommendations=2):
    # Exception Handling & Check Customer exists
    try:
        customer_data = order_producs.loc[[order_id]]  # Use double brackets to preserve DataFrame structure  
    except ValueError:
        return 'No Item Found'
    # Get distances and indices of neighbors  
    distances, indices = model.kneighbors(customer_data, n_neighbors=n_recommendations + 1)  
    
    # Remove self (0 distance) from the neighbors  
    indices = indices[0][distances[0] > 0]
    
    new_order = order_producs.loc[order_id]
    new_order_categories = Products.loc[new_order > 0, 'Category']

    similar_products = order_producs.iloc[indices.tolist()]  
    
    # Weighting system based on categories
    similar_products = find_categories(similar_products, new_order_categories.unique().tolist())
    
    aggregate_items = similar_products.sum(axis=0)
    
    recommended_items = aggregate_items.loc[new_order == 0].sort_values(ascending=False).head(n_recommendations)
    return Products.loc[Products.index.isin(recommended_items.index.tolist())]

row = np.random.choice(order_producs.index, 1)[0]
order_id = order_producs.loc[row].name
product_list = []
for product_id, item in zip(order_producs.loc[row].index, order_producs.loc[row]):
    if item > 0:
        product_list.append(product_id)

recommend_list = recommend_item(order_id,2)

print('Purchased Products:')
display(Products.loc[Products.index.isin(product_list)])
print('Recommended Products:')
display(recommend_list)


Purchased Products:


,Product Name,Category
Product ID,,
OFF-PA-10001166,Xerox 2,Office Supplies
TEC-MA-10001972,Okidata C331dn Printer,Technology
OFF-PA-10000587,"Array Parchment Paper, Assorted Colors",Office Supplies
OFF-BI-10000756,Storex DuraTech Recycled Plastic Frosted Binders,Office Supplies
FUR-BO-10001608,"Hon Metal Bookcases, Black",Furniture
OFF-ST-10004123,Safco Industrial Wire Shelving System,Office Supplies
TEC-PH-10000369,HTC One Mini,Technology
OFF-PA-10001166,Xerox 1932,Office Supplies


Recommended Products:


,Product Name,Category
Product ID,,
FUR-TA-10002607,KI Conference Tables,Furniture
OFF-PA-10003177,Xerox 1999,Office Supplies
